In [2]:
import pandas as pd
from tqdm import tqdm
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import numpy as np
#import unidecode
import string

#Modeling
%config InlineBackend.figure_format = 'svg'
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, precision_recall_curve,f1_score, fbeta_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.metrics import accuracy_score
#from mlxtend.classifier import StackingClassifier
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import StackingClassifier
from xgboost import XGBClassifier
from sklearn import tree
from sklearn.tree import plot_tree
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import GridSearchCV
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, StratifiedKFold, cross_val_score, cross_val_predict
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score, roc_auc_score,log_loss, confusion_matrix, precision_score, recall_score, accuracy_score 
from sklearn import linear_model, ensemble , tree 
from sklearn.tree import DecisionTreeClassifier

# Libraries for text preprocessing
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
nltk.download('wordnet') 
from nltk.stem.wordnet import WordNetLemmatizer

#Word cloud
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.feature_extraction.text import CountVectorizer
import re

#Barplot of most freq Bi-grams
import seaborn as sns

from sklearn.feature_extraction.text import TfidfTransformer

from scipy.sparse import coo_matrix

#removing html tags and text
from lxml import html

# getting ngrams
import nltk
from nltk import word_tokenize
from nltk.util import ngrams


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


#  Imports Libraries

# Explore Data

In [3]:
df = pd.read_csv('abstract______.csv', ) #read the data 
df.head(10)

,level_0,abstract,first_n_a
0,0,"L’esatta incidenza dell’ ARDS non è nota, poic...","Larsen, Reinhard"
1,1,Die Intensivmedizin in der Allgemeinchirurgie ...,"Larsen, Reinhard"
2,2,"Postoperative respiratorische Insuffizienz, Ri...","Larsen, Reinhard"
3,3,It has been said that streaming 30 minutes of ...,"Le Page, Michael"
4,4,Infections that cross over from other species ...,"Le Page, Michael"
5,5,"Last week, the Drug Enforcement Administration...","Knopf, Alison"
6,6,"Last month, the American Medical Association (...","Knopf, Alison"
7,7,When we visited Damian Family Care Centers' Wa...,"Knopf, Alison"
8,8,Are people drinking more now that they are loc...,"Knopf, Alison"
9,9,The study showing that Alcoholics Anonymous as...,"Knopf, Alison"


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 711 entries, 0 to 710
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   level_0    711 non-null    int64 
 1   abstract   711 non-null    object
 2   first_n_a  711 non-null    object
dtypes: int64(1), object(2)
memory usage: 16.8+ KB


In [5]:
df.drop(columns=['level_0'],axis=0,inplace=True)

In [6]:
df.columns

Index(['abstract', 'first_n_a'], dtype='object')

In [7]:
df.describe()

,abstract,first_n_a
count,711,711
unique,563,10
top,"Emeritus Professor Alan Glasper, from the Univ...","Canady, Valerie A."
freq,3,131


# NLP Preprocessing

### 1. Text Preprocessing

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 711 entries, 0 to 710
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   abstract   711 non-null    object
 1   first_n_a  711 non-null    object
dtypes: object(2)
memory usage: 11.2+ KB


In [41]:
from IPython.display import Markdown, display
for i in range(10):
    display(Markdown('> '+df['abstract'][i]))
    print('--------------------------------------')

> l’esatta incidenza dell’ ards non è nota poiché nella maggior parte delle diagnosi non viene impiegata una definizione univoca si può comunque affermare che sussiste una frequenza di circa – casi di malattia per abitanti

--------------------------------------


> die intensivmedizin der allgemeinchirurgie umfasst v die behandlung akuter abdominalerkrankungen wie z b der akuten pankreatitis gastrointestinaler blutungen ileus und peritonitis sowie postoperativer komplikationen bei bauchchirurgischen eingriffen wie z b peritonitis ileus oder nachblutungen hinzu kommen die postoperative intensivüberwachung und behandlung größerer operationen bei denen der frühphase bestimmte komplikationen zu erwarten sind nicht selten treten zu den spezifischen risiken allgemeinchirurgischer eingriffe postoperative störungen einzelner oder mehrerer organe hinzu und komplizieren den postoperativen verlauf

--------------------------------------


> postoperative respiratorische insuffizienz rippenserienfrakturen lungenkontusion akutes lungenversagen pulmonale aspiration von magensaft pneumonie und akut dekompensierte copd gehören zu den häufigsten erkrankungen die eine intensivmedizinische behandlung mit apparativer unterstützung der atmung erfordern bei schwersten formen – v bei ards – ist ein erheblicher apparativer und personeller aufwand erforderlich nicht selten trotz aller maßnahmen mit tödlichem ausgang

...

--------------------------------------


> people drinking locked house nearest dearest facing job loss lost job bored stressed probably drinking healthy way coping

--------------------------------------


> study showing alcoholic anonymous effective effective psychotherapy see “research aa better pathway abstinence psychotherapy” adaw march published day covid‐ federal state restriction group gathering “social distancing” took effect making aa meeting untenable seems like cruel timing john kelly phd one author study provided infographic adaw meeting keith humphreys phd co‐author quoted recently abc news saying concerned patient extreme mental health issue addiction face‐to‐face contact key

--------------------------------------


In [10]:
df['abstract']

0      L’esatta incidenza dell’ ARDS non è nota, poic...
1      Die Intensivmedizin in der Allgemeinchirurgie ...
2      Postoperative respiratorische Insuffizienz, Ri...
3      It has been said that streaming 30 minutes of ...
4      Infections that cross over from other species ...
                             ...                        
706    More than 100 coronavirus vaccines are in deve...
707    Death toll approaches half a million, but pand...
708    The outcome of a consultation on mandatory COV...
709    A lack of clear testing data means researchers...
710    Good ventilation is essential in indoor spaces...
Name: abstract, Length: 711, dtype: object

In [11]:
df['abstract']= df['abstract'].apply(lambda x: x.lower())

In [12]:
df['abstract'] = df['abstract'].str.replace('\d+', '')

<ipython-input-12-9f5a5a38dfa1>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['abstract'] = df['abstract'].str.replace('\d+', '')


In [13]:
#remove punctuation
#df['body_text'] = df['body_text'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '' , x))

In [14]:
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

df['abstract'] = df['abstract'].apply(remove_punctuations)

In [15]:
#remove_URL
def remove_URL(text):
    return re.sub(r"http\S+", "", text)

df['abstract'] = df['abstract'].apply(remove_URL)


In [16]:
custom_stop_words = [
    'doi', 'preprint', 'copyright', 'peer', 'reviewed', 'org', 'https', 'et', 'al', 'author', 'figure', 
    'rights', 'reserved', 'permission', 'used', 'using', 'biorxiv', 'medrxiv', 'license', 'fig', 'fig.', 
    'al.', 'Elsevier', 'PMC', 'CZI'
]

#remove stopwords
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(custom_stop_words)

def remove_stopwords(text):
   
    return " ".join([word for word in text.split() if word not in stop_words])
df['abstract'] = df['abstract'].apply(remove_stopwords)


In [17]:
stop_words

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [18]:
df['abstract']

0      l’esatta incidenza dell’ ards non è nota poich...
1      die intensivmedizin der allgemeinchirurgie umf...
2      postoperative respiratorische insuffizienz rip...
3      said streaming minutes netflix releases much c...
4                infections cross species deadly problem
                             ...                        
706                coronavirus vaccines development work
707    death toll approaches half million pandemic st...
708    outcome consultation mandatory covid vaccinati...
709    lack clear testing data means researchers stru...
710    good ventilation essential indoor spaces inclu...
Name: abstract, Length: 711, dtype: object

In [19]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

df["abstract"] = df["abstract"].apply(lambda text: lemmatize_words(text))

In [20]:
#def tokenization(text):
    #nltk_tokens = nltk.word_tokenize(text)
    #return nltk_tokens

#df['abstract']= df['abstract'].apply(lambda x: tokenization(x))

In [21]:
df.first_n_a.unique()

array(['Larsen, Reinhard', 'Le Page, Michael', 'Knopf, Alison',
       'Canady, Valerie A.', 'Vaughan, Adam', 'Sutton, Halley',
       'Foster, Sam', 'Glasper, Alan', 'Lawton, Graham', 'Price, Sean'],
      dtype=object)

In [22]:
df.first_n_a.value_counts()

Canady, Valerie A.    131
Knopf, Alison          94
Lawton, Graham         77
Larsen, Reinhard       75
Vaughan, Adam          71
Le Page, Michael       70
Sutton, Halley         56
Glasper, Alan          47
Price, Sean            46
Foster, Sam            44
Name: first_n_a, dtype: int64

In [23]:
df['first_n_a'] = df['first_n_a'].map({'Larsen, Reinhard':1,'Le Page, Michael':2,
                                       'Knopf, Alison':3,'Canady, Valerie A.':4,
                                      'Vaughan, Adam':5,'Sutton, Halley':6,
                                      'Foster, Sam':7,'Glasper, Alan':8,
                                       'Lawton, Graham':9,'Price, Sean':10,})

# Split X & Y

In [24]:
from sklearn.model_selection import train_test_split
x = df['abstract']  
y = df['first_n_a']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [25]:
print(f'Shape of train:', x_train.shape)
print(f'Shape of test:', x_test.shape)

Shape of train: (568,)
Shape of test: (143,)


In [26]:
print(f'Shape of train:', y_train.shape)
print(f'Shape of test:', y_test.shape)

Shape of train: (568,)
Shape of test: (143,)


In [27]:
#df_train, df_test = train_test_split(df ,test_size=0.2,random_state=3 )
#-----------------------------------------------------------
#x_train = df_train.drop('first_n_a', axis = 1)
#y_train = df_train['first_n_a']
#-----------------------------------------------------------
#x_test = df_test.drop('first_n_a', axis = 1)
#y_test = df_test['first_n_a']

In [28]:
y_train.value_counts()    

4     105
3      79
9      61
2      58
5      56
1      53
6      47
8      39
10     37
7      33
Name: first_n_a, dtype: int64

In [29]:
y_test.value_counts()

4     26
1     22
9     16
3     15
5     15
2     12
7     11
6      9
10     9
8      8
Name: first_n_a, dtype: int64

In [30]:
#------------------------------------------------------------------------

In [31]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()

x_train_counts = count_vect.fit_transform(x_train)
x_train_counts.shape

(568, 4199)

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

X_train_tfidf = vectorizer.fit_transform(x_train) # remember to use the original X_train set
X_train_tfidf.shape

(568, 4199)

In [33]:
print(f'Shape of train:', x_train.shape)
print(f'Shape of test:', x_test.shape)

Shape of train: (568,)
Shape of test: (143,)


# Logistic Regression

In [35]:
from sklearn.metrics import classification_report
log_cfl = LogisticRegression(C=0.95, random_state=0)

log_cfl.fit(X_train_tfidf, y_train)

y_predict = log_cfl.predict(X_train_tfidf)

# evaluate the model
print(classification_report(y_train, y_predict))
print(confusion_matrix(y_train, y_predict))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00        53
           2       1.00      1.00      1.00        58
           3       1.00      1.00      1.00        79
           4       0.99      1.00      1.00       105
           5       1.00      1.00      1.00        56
           6       1.00      1.00      1.00        47
           7       1.00      1.00      1.00        33
           8       1.00      1.00      1.00        39
           9       1.00      1.00      1.00        61
          10       1.00      0.97      0.99        37

    accuracy                           1.00       568
   macro avg       1.00      1.00      1.00       568
weighted avg       1.00      1.00      1.00       568

[[ 53   0   0   0   0   0   0   0   0   0]
 [  0  58   0   0   0   0   0   0   0   0]
 [  0   0  79   0   0   0   0   0   0   0]
 [  0   0   0 105   0   0   0   0   0   0]
 [  0   0   0   0  56   0   0   0   0   0]
 [  0   0   0   0   0  47 

# RandomForest

In [39]:
rf_cfl = RandomForestClassifier(max_depth=2)
rf_cfl.fit(X_train_tfidf, y_train)

y_predict = rf_cfl.predict(X_train_tfidf)

# evaluate the model
print(classification_report(y_train, y_predict))
print(confusion_matrix(y_train, y_predict))

              precision    recall  f1-score   support

           1       1.00      0.98      0.99        53
           2       1.00      0.86      0.93        58
           3       1.00      0.49      0.66        79
           4       0.35      1.00      0.51       105
           5       0.00      0.00      0.00        56
           6       1.00      0.13      0.23        47
           7       1.00      1.00      1.00        33
           8       1.00      0.77      0.87        39
           9       1.00      0.90      0.95        61
          10       0.00      0.00      0.00        37

    accuracy                           0.65       568
   macro avg       0.73      0.61      0.61       568
weighted avg       0.72      0.65      0.61       568

[[ 52   0   0   1   0   0   0   0   0   0]
 [  0  50   0   8   0   0   0   0   0   0]
 [  0   0  39  40   0   0   0   0   0   0]
 [  0   0   0 105   0   0   0   0   0   0]
 [  0   0   0  56   0   0   0   0   0   0]
 [  0   0   0  41   0   6 

C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Decision Tree

In [92]:
weights = np.linspace(0.05, 0.95, 20)

gsc = GridSearchCV(
    estimator=DecisionTreeClassifier(),
    param_grid={
        'class_weight': [{0: x, 1: 1.0-x} for x in weights]
    },
    scoring='f1',
    cv=3
)
grid_result = gsc.fit(x_train , y_train)

print("Best parameters : %s" % grid_result.best_params_)

C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 898, in fit
    super().fit(
  File "C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 156, in fit
    X, y = self._validate_data(X, y,
  File "C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\base.py", line 430, in _validate_data
    X = check_array(X, **check_X_params)
  File "C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\utils\validation.py", line 63, in inner_f
    return f(*args, **kwargs)
  File "C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\utils\validat

ValueError: could not convert string to float: 'primary role personal protective equipment ppe within healthcare environment either protect patient healthcare worker healthcare worker patient patient carers vector infectious disease hospital healthcare institution need protect frontline worker healthcare assistant patient infection covid'

In [93]:
Decision_Tree = DecisionTreeClassifier(max_depth = 10,random_state=10, criterion = "gini", splitter = 'random', max_leaf_nodes = 10, min_samples_leaf = 5,**grid_result.best_params_)
Decision_Tree.fit(x_train, y_train)

NameError: name 'grid_result' is not defined

In [42]:
print('f1_score for train: ',f1_score(y_train, Decision_Tree.predict(x_train)))
print('f1_score for test: ',f1_score(y_test, Decision_Tree.predict(x_test)))

ValueError: could not convert string to float: 'primary role personal protective equipment ppe within healthcare environment either protect patient healthcare worker healthcare worker patient patient carers vector infectious disease hospital healthcare institution need protect frontline worker healthcare assistant patient infection covid'

In [ ]:
#------------------------------------------------------------------